<a href="https://colab.research.google.com/github/charutomo/MLDA-DLW-Hackathon-2021/blob/main/Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from keras import optimizers
from keras import applications
from keras.models import Model
from keras.callbacks import History,ReduceLROnPlateau

In [ ]:
# from tensorflow.keras.applications.resnet import preprocess_input
# from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.applications.densenet import preprocess_input

In [ ]:
# loading training data
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                #    shear_range=0.2,
                                  #  zoom_range=0.4,
                                  #  rotation_range=30,
                                   )

valid_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
# test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
        '/content/drive/MyDrive/BedSet/train',
        target_size=(45, 45),
        class_mode='categorical')

validation_generator = valid_datagen.flow_from_directory(
    '/content/drive/MyDrive/BedSet/valid',
    shuffle=False,
    target_size=(45, 45),
    class_mode='categorical')

In [ ]:
from tensorflow.keras.applications import ResNet152, ResNet50, InceptionV3, DenseNet121, DenseNet201

In [ ]:
model = Sequential()
# create the base pre-trained model
model.add(DenseNet201(input_shape=(45, 45,3), weights='imagenet', include_top=False))
# add a global spatial average pooling layer
model.add(GlobalAveragePooling2D())
# let's add a fully-connected layer
model.add(Dense(1024, activation='relu'))
# and a logistic layer -- let's say we have 60 classes
model.add(Dense(len(train_generator.class_indices), activation='softmax'))

In [ ]:
# specify training loss function 
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),
              metrics=['accuracy'])

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=3, verbose = 1)

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath='/content/drive/MyDrive/Models/desnet201e30',
    monitor='val_loss',
    save_weights_only=False,
    save_best_only=True)

model.fit(x=train_generator, validation_data=validation_generator, epochs = 30, callbacks=[reduce_lr,model_checkpoint])

In [ ]:
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/BedSet/test',
    shuffle=False,
    target_size=(45, 45),
    class_mode='categorical')

In [ ]:
# test_generator.classes
# test_generator.class_indices
# test_generator.filenames

In [ ]:
train_generator.class_indices.keys()

In [ ]:
train_generator.class_indices

In [ ]:
top1 = 0
top2 = 0
top3 = 0
predict = model.predict(test_generator)

for i in range(len(predict)):
  result = np.argsort(-predict[i])
  truth = test_generator.classes[i]
  if truth in result[:3]: top3 += 1
  if truth in result[:2]: top2 += 1
  if truth in result[:1]: top1 += 1

top1 = top1/len(predict)
top2 = top2/len(predict)
top3 = top3/len(predict)
print(top1, top2, top3)

In [ ]:
from tensorflow import keras
model = keras.models.load_model('/content/drive/MyDrive/Models/desnet201e30')